In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df=pd.read_csv(r"D:\mlproject\auto-price-predictor\notebook\data\cardata.csv")

In [3]:
df.head(2)

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82.0,5,550000


In [4]:
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [3]:
df.drop('car_name',axis=1,inplace=True)
df.drop('Unnamed: 0',axis=1,inplace=True)
        

In [7]:
df.head(1)

,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000


In [4]:
num_features=df.select_dtypes(include=["number"]).columns
cat_features =df.select_dtypes(include=["object"]).columns

In [5]:
df[num_features].skew()

vehicle_age       0.833712
km_driven        28.172711
mileage           0.104961
engine            1.666467
max_power         2.485129
seats             2.039983
selling_price    10.047048
dtype: float64

In [6]:
skwed_data=['km_driven', 'engine', 'max_power','seats','selling_price']

In [7]:
def detect_outliers(col):
    percentile25 = df[col].quantile(0.25)
    percentile75 = df[col].quantile(0.75)
    print('\n ####', col , '####')
    print("percentile25",percentile25)
    print("percentile75",percentile75)
    iqr = percentile75 - percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr
    print("Upper limit",upper_limit)
    print("Lower limit",lower_limit)
    df.loc[(df[col]>upper_limit), col]= upper_limit
    df.loc[(df[col]<lower_limit), col]= lower_limit    
    return df


In [8]:
for col in skwed_data:
         detect_outliers(col)


 #### km_driven ####
percentile25 30000.0
percentile75 70000.0
Upper limit 130000.0
Lower limit -30000.0

 #### engine ####
percentile25 1197.0
percentile75 1582.0
Upper limit 2159.5
Lower limit 619.5

 #### max_power ####
percentile25 74.0
percentile75 117.3
Upper limit 182.25
Lower limit 9.050000000000011

 #### seats ####
percentile25 5.0
percentile75 5.0
Upper limit 5.0
Lower limit 5.0

 #### selling_price ####
percentile25 385000.0
percentile75 825000.0
Upper limit 1485000.0
Lower limit -275000.0


In [16]:
df[skwed_data].skew()

km_driven        0.617437
engine           0.684096
max_power        1.067229
seats            0.000000
selling_price    0.968836
dtype: float64

In [9]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [10]:
x.head(1)

,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796.0,46.3,5


In [11]:
y

0         120000
1         550000
2         215000
3         226000
4         570000
          ...   
15406     250000
15407     925000
15408     425000
15409    1225000
15410    1200000
Name: selling_price, Length: 15411, dtype: int64

In [20]:
for feature in cat_features:
    print(feature,':', x[feature].nunique())

brand : 32
model : 120
seller_type : 3
fuel_type : 5
transmission_type : 2


In [24]:
num_features = x.select_dtypes(exclude="object").columns
num_features

Index(['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats'], dtype='object')

In [28]:
!pip install category_encoders

  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/98/47/598b4bf0ccf6f02915e71bdd23fe846a27adc2d3ba734f2ba5215d8e44f5/category_encoders-2.6.4-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/82.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/82.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/82.0 kB ? eta -:--:--
   -------------- ------------------------- 30.7/82.0 kB 187.9 kB/s eta 0:00:01
   ---------------------------------------  81.9/82.0 kB 383.3 kB/s eta 0:00:01
   ---------------------------------------- 82.0/82.0 kB 329.2 kB/s eta 0:00:00


In [12]:
num_features = ['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
onehot_columns = ['seller_type','fuel_type','transmission_type']
binary_columns = ['brand', 'model']

from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
binary_transformer = BinaryEncoder()

preprocessor = ColumnTransformer( transformers=[
        ("OneHotEncoder", oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features),
        ("BinaryEncoder", binary_transformer, binary_columns)
    ]
)

x= preprocessor.fit_transform(x)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((12328, 29), (3083, 29))

In [31]:
df.head(1)

,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796.0,46.3,5,120000


In [14]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor,VotingRegressor


In [41]:
LinearRegression_model = LinearRegression()
Ridge_model = Ridge()
Lasso_model = Lasso()
KNeighbors_model = KNeighborsRegressor()
DecisionTree_model = DecisionTreeRegressor()
SVR_model = SVR()
RF_model = RandomForestRegressor()

voting_regressor = VotingRegressor(estimators=[
    ("lr", LinearRegression_model),
    ("ridge", Ridge_model),
    ("dt", DecisionTree_model),
    ("rf", RF_model)
])

voting_regressor.fit(x_train, y_train)

y_pred = voting_regressor.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2s = r2_score(y_test, y_pred)
mae=mean_absolute_error(y_test,y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2s}')
print(f'mean absolute error:{mae}')

Mean Squared Error: 12472141324.505083
R^2 Score: 0.9145261471429001
mean absolute error:82620.67183356165


In [34]:
mse

12494431866.200624

In [35]:
rmse=np.sqrt(mse)

In [36]:
rmse

111778.49464991297

In [37]:
r2s

0.914373386006587

In [38]:
np.mean(y)

655763.6671857764

In [39]:
 rmse/np.mean(y)*100

17.04554555905983

In [43]:
models = {
    "LinearRegression_model": LinearRegression(),
    "Ridge_model": Ridge(),
    "Lasso_model": Lasso(),
    "KNeighbors_model": KNeighborsRegressor(),
    "DecisionTree_model": DecisionTreeRegressor(),
    "SVR_model": SVR(),
    "RF_model":RandomForestRegressor()
}
results = {}
for name, model in models.items():
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)  
    r2 = r2_score(y_test, y_pred)  
    mae=mean_absolute_error(y_test,y_pred)
    
    results[name] = {"MSE": mse, "R²": r2,"MAE":mae}

for model_name, metrics in results.items():
    print(f"{model_name} -->> MSE: {metrics['MSE']:.4f}, R²: {metrics['R²']:.4f}, MAE: {metrics['MAE']:.4f}")

LinearRegression_model -->> MSE: 22923926892.6400, R²: 0.8429, MAE: 115256.5693
Ridge_model -->> MSE: 22917734516.3861, R²: 0.8429, MAE: 115249.0656
Lasso_model -->> MSE: 22916553702.9264, R²: 0.8429, MAE: 115237.3416
KNeighbors_model -->> MSE: 10370870209.4145, R²: 0.9289, MAE: 68385.8985
DecisionTree_model -->> MSE: 14389038985.4157, R²: 0.9014, MAE: 78164.2838
SVR_model -->> MSE: 155345111837.4861, R²: -0.0646, MAE: 289831.1317
RF_model -->> MSE: 9746360290.4392, R²: 0.9332, MAE: 65793.5492


In [15]:
from sklearn.model_selection import GridSearchCV

In [45]:
KNeighbors = KNeighborsRegressor()
KNeighbors.fit(x_train,y_train)

KNeighborsRegressor()

In [46]:
KNeighbors.score(x_train,y_train)*100 , KNeighbors.score(x_test,y_test)*100

(95.41109174546443, 92.89265402615415)

In [48]:
p_g = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

In [49]:
gd=GridSearchCV(KNeighborsRegressor(),param_grid=p_g)
                  
gd.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 10],
                         'weights': ['uniform', 'distance']})

In [50]:
gd.best_params_

{'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}

In [16]:
KNeighbors = KNeighborsRegressor(metric= 'euclidean', n_neighbors= 7, weights= 'uniform')
KNeighbors.fit(x_train,y_train)

KNeighborsRegressor(metric='euclidean', n_neighbors=7)

In [17]:
y_pred=KNeighbors.predict(x_test)
mse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred)  
mae=mean_absolute_error(y_test,y_pred)
print(mse)
print(r2)
print(mae)

  File "C:\Users\Hp\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 199, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Hp\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Hp\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


10366933905.258097
0.9289535164698375
68757.37338399517


In [90]:
KNeighbors.score(x_train,y_train)*100 , KNeighbors.score(x_test,y_test)*100

(94.84002393839864, 92.89535164698376)

In [18]:
from xgboost import XGBRegressor

In [19]:
xgb_model = XGBRegressor()
xgb_model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [58]:
xgb_model.score(x_train,y_train)*100 , xgb_model.score(x_test,y_test)*100

(97.0818040914179, 94.13671174771733)

In [74]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200,300]}

In [61]:
gd=GridSearchCV(XGBRegressor(),param_grid=param_grid)
gd.fit(x_train,y_train)

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...ne,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]})

In [70]:
gd.best_params_

{'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 200}

In [20]:
xgb_model = XGBRegressor(learning_rate=0.05, max_depth=7, n_estimators=200)
xgb_model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
y_pred=xgb_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)  
r2 = r2_score(y_test, y_pred) 
mae=mean_absolute_error(y_test,y_pred)
print(mse)
print(r2)
print(mae)

8674356244.654852
0.9405530590140971
62705.84350166234


In [77]:
xgb_model.score(x_train,y_train)*100 , xgb_model.score(x_test,y_test)*100

(96.56772110581612, 94.05530590140971)

In [22]:
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(xgb_model, x, y, cv=5)  
print("Cross-validation scores:", cross_val_scores)
print("Mean cross-validation score:", cross_val_scores.mean())


Cross-validation scores: [0.94717455 0.92854256 0.92859687 0.94436783 0.92981191]
Mean cross-validation score: 0.9356987450608992


In [23]:
cross_val_scores = cross_val_score(KNeighbors, x, y, cv=5)  
print("Cross-validation scores:", cross_val_scores)
print("Mean cross-validation score:", cross_val_scores.mean())

Cross-validation scores: [0.93297408 0.92605485 0.9213329  0.93111265 0.91542348]
Mean cross-validation score: 0.9253795923857056
